In [21]:
!gcc -o ejecutable algortimo.c

algortimo.c: In function ‘escribir_salida’:
algortimo.c:223:20: warning: ‘return’ with a value, in function returning void
  223 |             return 1;
      |                    ^
algortimo.c:217:6: note: declared here
  217 | void escribir_salida(double salida, int flag){
      |      ^~~~~~~~~~~~~~~
algortimo.c:231:20: warning: ‘return’ with a value, in function returning void
  231 |             return 1;
      |                    ^
algortimo.c:217:6: note: declared here
  217 | void escribir_salida(double salida, int flag){
      |      ^~~~~~~~~~~~~~~
algortimo.c: In function ‘insercion_reverberacion’:
algortimo.c:244:16: warning: ‘return’ with a value, in function returning void
  244 |         return 1;
      |                ^
algortimo.c:240:6: note: declared here
  240 | void insercion_reverberacion(){
      |      ^~~~~~~~~~~~~~~~~~~~~~~
algortimo.c:254:29: warning: implicit declaration of function ‘strtol’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#ind

In [23]:
import wave, subprocess

def audio_to_q114(input_file, Qformat_file):
  # Abre el archivo WAV y toma sus datos de audio
    with wave.open(input_file, 'rb') as wav_in:
        paramsIn = wav_in.getparams()
        header_size = paramsIn.nframes * paramsIn.sampwidth
        audio_data = wav_in.readframes(paramsIn.nframes)
  # Convierte esos datos a entero
    binary_data = bytearray(audio_data)
    normalized_float_data = []
    int_data = [int(byte) for byte in binary_data]
  # Normaliza los datos entre -1 y 1
    for value in int_data:
        normalized_value = (value / 255.0) * 2 - 1
        normalized_float_data.append(normalized_value)
  # Escribe el formato Q1.14
    qformatData = []
    for val in normalized_float_data:
        if val < 0:
            binary = "1"  # Signo negativo
        else:
            binary = "0"  # Signo positivo

        integer = int(abs(val))
        binary = binary + str(integer) # Parte entera
  # Sub-algoritmo para convertir el float a binario
        decimal = abs(val) - int(abs(val))
        sum = 0
        for i in range(14):
            if sum + 2**(-(i + 1)) < decimal:
                sum += 2**(-(i + 1))
                binary = binary + "1"
            else:
                binary = binary + "0"
        while len(binary) < 15:
            binary += "0"
        qformatData.append(binary) # Parte decimal
  # Escritura de los datos Q1.14 a un archivo TXT
    with open(Qformat_file, 'w') as txt_out:
        for value in qformatData:
            txt_out.write(value + "\n")
    return paramsIn

def q114_to_audio(Qformat_file, reconstructed_file, paramsIn):
  # Lee los datos del TXT generado antes
    with open(Qformat_file, 'r') as txt_in:
        qformat_data = [line.strip() for line in txt_in]
  # Convierte los datos de Q1.14 a float
    float_data = []
    for binary in qformat_data:
        sign_bit = int(binary[0])
        integer_part = int(binary[1], 2)
        fractional_part = 0
        for i in range(14):
            fractional_part += int(binary[i + 2]) * (2 ** -(i + 1))
        value = (-1 if sign_bit == 1 else 1) * (integer_part + fractional_part)
        float_data.append(value)
  # Convierte los float a binario
    int_data = [int((value + 1) * 127.5) for value in float_data]
    binary_data = bytearray(int_data)
  # Escribe los datos en binario al nuevo archivo WAV
    with wave.open(reconstructed_file, 'wb') as wav_out:
        wav_out.setparams(paramsIn)
        wav_out.writeframes(binary_data)

def floatToBinary_and_audioReconstruction(input_file, reconstructed_file, paramsInput):
    with open(input_file, 'r') as txt_in:
        float_data = [float(line.strip()) for line in txt_in]

    int_data = [(int((value + 1) * 127.5)) for value in float_data]
    binary_data = bytearray(int_data)

    with wave.open(reconstructed_file, 'wb') as wav_out:
        wav_out.setparams(paramsInput)
        wav_out.writeframes(binary_data)

def main():
  paramsInput = audio_to_q114("audio_mono16k.wav", "input.txt")
  subprocess.run(["./ejecutable"])
  floatToBinary_and_audioReconstruction("insercion.txt", "reverbed_audio.wav", paramsInput)
  floatToBinary_and_audioReconstruction("reduccion.txt", "desreverbed_audio.wav", paramsInput)

main()